In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

import sys
sys.path.append("../esg")
import preproccessing as prep


### TODO
refactor 

In [15]:
""" spec = [
    ('initial_value', numba.float64[:]),
    ('inflows', numba.float64[:]),
    ('returns', numba.float64[:,:,:]),
    ('capital', numba.float64[:])
]
@jitclass(spec)  """
class Simulation():
    def __init__(self, initial_value, inflows, returns) -> None:
        self.initial_value = np.full(returns.shape[1], initial_value)
        self.returns = returns
        self.inflows = inflows

    def step(self, current_value,inflow,weights,returns):
        portfolio_return = returns @ weights
        next_assets_value =  (current_value + inflow) * (1+portfolio_return)
        return next_assets_value

    def run(self, strategy):
        self.capital = self.initial_value
        T = len(strategy)
            
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],strategy[t],self.returns[t])


class MaxPercentile():

    def __init__(self, percentile) -> None:
        self.percentile = percentile

    def __call__(self, values: np.array) -> float:
        #return np.round(np.percentile(values,self.percentile))
        return np.percentile(values,self.percentile)

class Fitness:
    def __init__(self,simulator: Simulation,portfolios,objective) -> None:
        self.simulator = simulator
        self.portfolios = portfolios
        self.objective = objective

    def __call__(self, solution: np.array) :
        strategy = np.take(self.portfolios,solution, axis=0)
        self.simulator.run(strategy)
        result = self.objective(self.simulator.capital)
        return result
    
def glide_path_grid_search(T, capital, inflows,risk, portfolios, returns):
    l = np.arange(T,0,-1)
    capital = np.repeat(capital,returns.shape[1])

    glide_paths = np.full(T+1,len(portfolios))
    perc = np.zeros((l.shape[0],len(portfolios)))

    objective = MaxPercentile(risk)
    simulator = Simulation(capital,inflows,returns)
    fitness = Fitness(simulator,portfolios,objective)

    for t in l:

        for portfolio in range(len(portfolios)):

            solution = np.full(t,portfolio)
            
            perc[t-1,portfolio] = fitness(solution)
        glide_paths[t-1] = np.argmax(perc[t-1])
    
    return np.flip(glide_paths)[1:]

#optimize glide with resamping
def resampled_glide_path_grid_search(T,capital, inflows, returns, risk, portfolios, number_of_runs):
    batches = np.asarray(np.array_split(returns,number_of_runs,axis=1))
    glides = np.zeros((number_of_runs,T))


    for batch in range(len(batches)):
        glide = glide_path_grid_search(T,capital,inflows,risk,portfolios,batches[batch])
        glides[batch] = glide

    glides = np.int32(glides)

    result = np.zeros(T)
    for t in range(T):
        result[t] = np.argmax(np.bincount(glides[:,t], minlength=11))
        
    return pd.Series(result, index=range(T,0,-1))

In [100]:
def expected_returns(period):
    return pd.Series({
		'ACWI': 0.07/period,
		'^TBSP': 0.035/period,
		'XAUPLN': 0.05/period,
		'MWIG40': 0.065/period,
		'IHYU.L': 0.038/period,
		'PLOPLN3M': 0.03/period,
		'edo': 0.025/period
})
#acwi and edo
corr = np.array([[1,0],[0,1]])
std = np.array([0.135,0.02])
cov_matrix = prep.corr_to_cov(corr, std)

assets = ['ACWI', 'edo']
means = expected_returns(1)[assets]
number_of_scenarios = 100 * 2500
T = 50
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov_matrix, T))

#calculate inflation
sim_returns = np.expm1(sim_returns)
sim_returns[:,:,1] = np.where(sim_returns[:,:,1]<0,0,sim_returns[:,:,1])
sim_returns[:,:,1] = sim_returns[:,:,1] + 0.0123

sim_returns = np.array(sim_returns).transpose(1,0,2)
sim_returns.shape

(50, 250000, 2)

In [5]:
""" portfolios = [[0.0,1.0],
             [0.2,0.8],
             [0.4,0.6],
             [0.6,0.4],
             [0.8,0.2],
             [1.0,0.0]]  """

portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]]



In [104]:
number_of_runs = 100
T = 50
capital = 10.0
risks = [5,15,25,35,45]
#risks = [10,20,30,40,50]
inflows = np.cumproduct(np.full(T,1.025))*5

glide_paths = np.zeros((T,len(risks)))

for risk in range(len(risks)):
    glide_paths[:,risk] = resampled_glide_path_grid_search(T,capital,inflows,sim_returns,risks[risk],portfolios,number_of_runs)

glide_paths = pd.DataFrame(data=np.int32(glide_paths),index=range(T,0,-1), columns=risks)
glide_paths

,5,15,25,35,45
50,5,10,10,10,10
49,5,10,10,10,10
48,5,10,10,10,10
47,5,10,10,10,10
46,5,10,10,10,10
45,4,10,10,10,10
44,4,10,10,10,10
43,4,10,10,10,10
42,4,10,10,10,10
41,4,10,10,10,10


In [105]:
glide_paths.to_excel('glide_paths_norm_29.10.2022.xlsx')

In [110]:
#TODO sim returns for glide paths

inflows = np.cumproduct(np.full(T,1.025))*0
initial_capital = 1.0
simulator = Simulation(initial_capital,inflows,sim_returns)

glide_returns = np.zeros((T,len(risks)))

for t in range(0,T,):
    for risk in range(len(risks)):
        glide_path = glide_paths.loc[t+1:,risks[risk]]
        fitness = Fitness(simulator,portfolios,MaxPercentile(risks[risk])) 
        glide_returns[t,risk] = fitness(glide_path)         
      
glide_returns = pd.DataFrame(data=np.flip(glide_returns,axis=0),index=range(T,0,-1),columns=risks)
glide_returns

,5,15,25,35,45
50,6.550221,9.513329,15.179212,22.096855,29.368334
49,6.278023,8.987451,14.251275,20.678387,27.442717
48,6.025883,8.491508,13.375426,19.372067,25.603785
47,5.787623,8.024964,12.573735,18.104648,23.909413
46,5.558296,7.593658,11.812526,16.959287,22.314870
45,5.338783,7.178153,11.096221,15.882900,20.863564
44,5.125853,6.796381,10.423099,14.884202,19.474645
43,4.916810,6.432426,9.790042,13.933675,18.180536
42,4.724378,6.089898,9.209766,13.044589,16.974015
41,4.535780,5.769194,8.648123,12.216365,15.853240


In [115]:
description = pd.DataFrame(data={'Opis': """"
Numer_of_scenarios = 250000,
number_of_runs = 100
T = 50
capital = 10.0
risks = [5,15,25,35,45]
inflows = np.cumproduct(np.full(T,1.025))*5
"""},index=[1])

In [117]:
with pd.ExcelWriter('glide_paths_norm_29.10.2022.xlsx', engine='openpyxl', mode='w') as writer:  
    glide_paths.to_excel(writer, sheet_name='Glide_paths')
    glide_returns.to_excel(writer, sheet_name='Returns')
    description.to_excel(writer,sheet_name='Description')
